_[Please follow the instructions [here](https://github.com/Hermi999/bce) to see how to properly use this bash-kernel Notebook]_

__NOTE TO GOOGLERS:__ The Context-aware access part of this tutorial will not work in the __google.com GCP Organization__, as you do not have the rights to view/edit the access policies and levels.


# Beyond Corp - Walkthrough

The following notebook is used to guide you through setting up Beyond Corp on Google Cloud Platform (GCP). 
It contains the following use cases:
- BeyondCorp (partial) [Gitops-setup](#Creating-access-levels) __[ready]__
- BeyondCorp for [Google App Engine web applications](#Deploy-a-first-sample-application) __[ready]__
- BeyondCorp for [Compute Engine web applications](#Setup-BeyondCorp-for-GCE-Apps) __[ready]__
- BeyondCorp for [GKE web applications](#Setup-BeyondCorp-for-GKE-Apps) __[work in progress]__
- BeyondCorp for internal on-premise (or other Cloud) web applications __[work in progress]__
- BeyondCorp for external on-premise (or other Cloud) web applications __[work in progress]__
- BeyondCorp for TCP forwarding __[work in progress]__

## Context
As more employees work from home, it is important to provide more options for them to securely access internal services and sites.
These sites may be your intranet portal, sensitive HR or financial systems that are not exposed directly externally. Traditionally, companies accessed these sites after establishing a VPN tunnel from an employee’s laptop directly to the corporate network. Just about any administrator of such a system will tell you VPNs are expensive to procure, require a lot of resources, quite cumbersome to maintain and essentially provide security based on dynamically expanding the trusted network.

At Google, we found the access model based primarily on a network perimeter did not sufficiently scale with a mobile workforce nor provide selective security guarantees for sensitive applications. Our approach was to assess the overall context of a given request for an application. By that we would take signals derived from multiple sources at runtime to evaluate the integrity of a call against the resource being accessed. For example, the evaluation signals for a request would include who is making the call, from what device, from what network, at what time, and so on. A request is not simply evaluated using an administrator defined static if-then chain, but also assigned trust levels based on heuristics inferred from prior contexts.

The internal system Google used to achieve this is described in six BeyondCorp research papers which goes into more details about these signals. For our customers we make this same system available as __BeyondCorp (BCE).__

As you can see on the image below, BCE consists of multiple components:
- Cloud Identity
- Identity Aware proxy
- Google Cloud Load Balancer
- Access Context Manager
- IAP Connector

![BCE](https://github.com/Hermi999/bce/blob/master/img/bce.png?raw=true)

## How to use this colab notebook

*   If you didn't start at following Github Repository, please start there to see how you can use this bash-kernel based notebook: https://github.com/Hermi999/bce
*   **You can use this notebook either as a reference and copy the commands directly into GCP Cloud Shell, or you use it with a Jupyter Notebook with a bash kernel.**
*   If you like to execute a block use STRG+ENTER
*   If you just want to run highlighted commands use STRG+SHIFT+ENTER

## Getting ready
Let's start by checking the version of the gcloud sdk. Click into the next line and press STRG+ENTER

In [ ]:
gcloud version

We do not need to authenticate ourselfs to run gcloud commands. It automatically uses the Compute Engine Service Account which was copied onto the VM during creation.

You can check this by running the command:

In [ ]:
gcloud auth list

This Service Account should have all necessary rights to proceed.

We also double check if we are in the right project:

In [ ]:
gcloud projects list

If the project is wrong you can set your project with 'gcloud config set project [project_name].

## Deploy a first sample application
We start with a first sample application on GCP. 
For this we will use the __App Engine__, which was the first GCP service (2008) and is a PaaS for hosting web applications. 

Our Git repository we cloned onto this machine contains an App Engine sample code. 

In [ ]:
cd ~/bce/samples/appengine
ls

Now we can deploy the simple sample to App Engine:

In [ ]:
gcloud app create --region=europe-west3

Wait until you see the message gcloud app browse

In [ ]:
gcloud app deploy --quiet

In [ ]:
gcloud app browse

When you follow the link you should see an empty website with a simple sign in link.
Now that we have our app running it's time to secure it with Identity-aware proxy.


## Configure OAuth 


### The Consent Screen (=Brand)
(https://cloud.google.com/iap/docs/programmatic-oauth-clients)

Because IAP accesses Google users’ data (email address) you need to configure the OAuth.
This needs to be done once per GCP Project. Typically you manage multiple on-prem or AWS/Azure web apps from the same projects, so it only need to be done rearily. 

The OAuth consent screen, which contains branding information for users, is known as a brand. Brands can be limited to internal users or public users. An __internal brand__ makes the OAuth flow accessible to someone who belongs to the same Google Workspace organization as the project. A __public brand__ makes the OAuth flow available to anyone on the internet. There is only one brand per GCP project.

OAuth API verification is a little bit difficult to understand - read more about it here: https://support.google.com/cloud/answer/9110914

Let's create our brand:

In [ ]:
APP_TITLE=IAP_TEST
SUPPORT_EMAIL=[enter your email]

gcloud alpha iap oauth-brands create --application_title=$APP_TITLE --support_email=$SUPPORT_EMAIL

The support email displayed on the OAuth consent screen. This email address can either be a user's address or a Google Groups alias.

Now we can list our new brand and extract the Brand-ID:

In [ ]:
gcloud alpha iap oauth-brands list
BRAND=$(gcloud alpha iap oauth-brands list --format="value(name)")

echo "BRAND: $BRAND"

### The OAuth Client
(https://cloud.google.com/iap/docs/programmatic-oauth-clients#creating_an_oauth_client)

Now we create our OAuth Client for the IAP. You configure one OAuth client per app.

In [ ]:
CLIENT_NAME=iap
PROJECT_ID=$(gcloud config get-value core/project)

gcloud alpha iap oauth-clients create $BRAND --display_name=$CLIENT_NAME
#gcloud alpha iap oauth-clients create projects/$PROJECT_ID/brands/1023990209690 --display_name=CLIENT_NAME

Be aware that it's only possible to create oauth clients via gcloud if the brand is __internal__. Otherwise you have to use the GUI.

Now we enable IAP for the web service. Fill in the client_id and secret from the previous step:

In [ ]:
CLIENT_ID=
SECRET=
gcloud alpha iap web enable --resource-type=app-engine --oauth2-client-id=$CLIENT_ID --oauth2-client-secret=$SECRET

Now we have activated IAP for the App Engine App. This means that we do not have access any more...

In [ ]:
gcloud app browse

## Creating access levels
https://cloud.google.com/access-context-manager/docs/overview

*__NOTE TO GOOGLERS:__ The Context-aware access part of this tutorial will not work in the google.com GCP Organization, as you do not have the rights to view/edit the access policies and levels.*


While Identity-Aware Proxy (IAP) lets you manage who has access to your web services, with __Access Context Manager__ you can define __when__ (in which circumstances) the request is allowed. Access Context Manager allows Google Cloud organization administrators to define fine-grained, attribute based access control for projects and resources in Google Cloud.

Administrators first define an __access policy__, which is an organization-wide container for __access levels__.
Access levels describe the necessary requirements for requests to be honored. Examples include:
- Device type and operating system
- IP address
- User identity
- Device Type (you need a BeyondCorp or Cloud Identity Premium license)


### Use GitOps for Access Levels
We could create access levels via the GCP Console UI, but that's too easy :) Instead we want to set up a GitOps-based approach for our access levels. This makes it easy to track changes and approve changes which can be submitted via a __merge request__.

#### Create the access policy
First we can [create an access policy](https://cloud.google.com/access-context-manager/docs/create-access-policy) for __Access Context Manager__. 1 Organization can only have 1 access policy:

In [ ]:
POLICY_TITLE="my_org_policy"
PROJECT_ID=$(gcloud config get-value core/project)
ORG_ID=$(gcloud projects describe $PROJECT_ID --format="value(parent.id)")

gcloud access-context-manager policies create --organization $ORG_ID --title $POLICY_TITLE

#### Define Access levels

https://cloud.google.com/access-context-manager/docs/create-basic-access-level

Now let's have a look at the example access context manager __access level policies__ I've prepared for you. For this exercise I just created 2 simple policies:

__Low Trust (lowtrust.yaml):__
Simply checks if the users accesses the app from one of the following 3 countries: Austria, Italy or Germany.

__High Trust (hightrust.yaml):__
Build upon the Low Trust policy, but additionally limits access to the CIDR: 80.110.0.0/16

In [ ]:
cat ~/bce/accesscontextmngr/lowtrust.yaml

In [ ]:
cat ~/bce/accesscontextmngr/hightrust.yaml

I also included a more complex example which includes specific device policies. We will not use this for our exercise here because you need to have a BeyondCorp license or Cloud Identity Premium. Still let's have a closer look at it:

In [ ]:
cat ~/bce/accesscontextmngr/complex.yaml

----------------------------------------------------------------
As you can see, the access level policies are easy to create, read and can also be combined.

We could now simply apply them with the following gcloud command:
````bash
gcloud access-context-manager levels create ...
````

But we don't do this now because we want to set up a Gitops-based pipeline to automatically deploy new access levels whenever the admin commits them to a git repository....

If you still want to try already (because you can't wait) it out here are the commands:

In [ ]:
ORG_ID=$(gcloud projects describe $PROJECT_ID --format="value(parent.id)")
POLICY_NAME=$(gcloud access-context-manager policies list --organization=$ORG_ID --format="value(NAME)")
cd ~/bce/accesscontextmngr
gcloud access-context-manager levels create "Low_Trust" --basic-level-spec=lowtrust.yaml --combine-function=OR --policy=$POLICY_NAME --title="Low Trust"
# replace POLICY_ID with actual ID of the policy
sed "s/POLICY_ID/$POLICY_NAME/g" hightrust.yaml > hightrust2.yaml
gcloud access-context-manager levels create "High_Trust" --basic-level-spec=hightrust2.yaml --combine-function=OR --policy=$POLICY_NAME --title="High Trust"

# UI
echo "https://console.cloud.google.com/security/access-level?organizationId=$ORG_ID"

In the UI it will look something like this:
![](https://github.com/Hermi999/bce/blob/master/img/accesscontextmanager.png?raw=true)

We can also export all access level policies, change them and then re-import them again:

In [ ]:
# Export all
gcloud access-context-manager levels list --policy $POLICY_NAME --format=json > all.json

Now you can open and modify the file...

... And then re-import all access level policies:

In [ ]:
gcloud access-context-manager levels replace-all $POLICY_NAME --source-file=all.json

If you created the access policies, let's clean up again:

In [ ]:
gcloud access-context-manager levels delete "High_Trust" --policy=$POLICY_NAME -q
gcloud access-context-manager levels delete "Low_Trust" --policy=$POLICY_NAME -q

#### Prepare access level renderer
Now that we know how to deploy, update and delete access level policies, let's put those commands into a file so that we can use it for an automated deployment via a Gitops-based approach.

While it's pretty straight forward to use gcloud to create, update and delete level policies, we have to think on how we can elegantly resolve levels which depend on each other. When we look at the hightrust.yaml file, we can see that the reference to the lowtrust level contains the policy ID:

````bash
...
requiredAccessLevels:
  - accessPolicies/POLICY_ID/accessLevels/Low_Trust
````

Now we could hard-code the Policy ID into our yaml files, but it's better to dynamically replace it before deploying the yaml to the environment. This makes sure that we don't need to adapt all our policy files when we use for another policy ID.

Another problem is that the the level yaml spec files can't include the "name", "title", "description" and "combine function" and that we need to provide this information via the _gcloud access-context-manager levels create_ command. --> The question is how/where we can store this information in Git? 

For the moment (maybe there will be improvement of Context-aware access in future) it seems like we need to put this information into our yaml files and parse and remove this information before deploying with _gcloud access-context-manager levels create_. Interestingly, when we show an existing level policy with  __gcloud access-context-manager levels describe__ the level policy is already shown in a format containing the above mentioned attributes: 

````bash
basic:
  combiningFunction: OR
  conditions:
  - ipSubnetworks:
    - 0.0.0.0/0
  - regions:
    - IT
    - DE
    - AT
description: Low Trust Access Level
name: accessPolicies/1082987218928/accessLevels/Low_Trust
title: Low Trust
````

But atm we can't re-import an access level exported via the describe command (at least not that I know of). Still we can use exactly this format (maybe it's later possible to import directly using this format) to store our access levels and then render them into the format which is necessary at the moment + the corresponding gcloud command. So the yaml above will turn into:

````bash
- ipSubnetworks:
  - 0.0.0.0/0
- regions:
  - IT
  - DE
  - AT
````

AND the following gcloud commands:

````bash
gcloud access-context-manager levels create accessPolicies/1082987218928/accessLevels/Low_Trust --basic-level-spec=lowtrust.yaml --combine-function=OR --title="Low Trust" --description="Low Trust access level"

gcloud access-context-manager levels update accessPolicies/1082987218928/accessLevels/Low_Trust --basic-level-spec=lowtrust.yaml --combine-function=OR --title="Low Trust" --description="Low Trust access level"

````

Implementing this is pretty easy - Here is the code for Python:


In [ ]:
cat ~/bce/accesscontextmngr/CD/p.py

It...
- parses all YAML files in the accesscontextmngr/CD folder
- renders them into the official level spec file and stores them in accesscontextmngr/CD/rendered
- creates the corresponding "gcloud" commands and stores them in the bash file create_update_levels.sh

Let's try it our here locally in our Jupyter environment:

In [ ]:
cd ~/bce/accesscontextmngr/CD
python p.py
echo "done"

Look at the resulting 3 files:

In [ ]:
echo "-------create_update_levels.sh--------"
cat create_update_levels.sh && echo "-------High_Trust.yaml--------"
cat rendered/High_Trust.yaml && echo "------Low_Trust.yaml---------"
cat rendered/Low_Trust.yaml

We can see that the gcloud commands contain POLICY_ID. The pipeline needs to replace this later with the acutual ID.

We delete the created files again before we continue:

In [ ]:
rm -r rendered
rm create_update_levels.sh
echo "done"

#### Prepare Cloud Build file

Now that we created a renderer for the access levels, we create the __cloudbuild.yaml__ file for [Google Cloud Build](https://cloud.google.com/cloud-build/). We use Cloud Build for automating our deployment, the cloudbuild.yaml file tells Cloud Build what to do. I've prepared it aleady for you:

In [ ]:
cd ~/bce/accesscontextmngr
cat cloudbuild.yaml

It's fairly simple and just consists out of 2 steps:
- execution of the python script (like we did above)
- execution of the bash script which contains the create/update policy commands. The complicated looking "sed" command simply replaces the POLICY_ID with the actual policy ID for this organization

You might have also noticed the __requirements.txt__ file. This one simple contains the dependencies for the p.py script:

In [ ]:
cat requirements.txt

#### Create the Continuious Deployment
For simplicity we will use [Cloud Source Repositories](https://cloud.google.com/source-repositories) for setting up our Git-based deployment, but you can use any SCM tool you like (e.g. Bitbucket, Gitlab, Github, etc.).

We start with copying the Access Level Config YAML files to a new folder and removing everything we don't need:

In [ ]:
cp -r ~/bce/accesscontextmngr ~
cd ~/accesscontextmngr/
rm complex.yaml hightrust*.yaml lowtrust.yaml test.yaml
echo "done"

Now let's create a new Cloud Source Repository:

In [ ]:
ACM_NAME="acm_staging"
gcloud source repos create $ACM_NAME

Now we initialize the new Git Repo and push the code from the local Git repository to the Cloud Source Repository we created in the previous step. Replace the __email__ and __name__:

In [ ]:
YOUR_EMAIL="h@hewagner.com"
YOUR_NAME="Hermann Wagner"
git init
git config --global user.email $YOUR_EMAIL
git config --global user.name $YOUR_NAME
git add -A
git commit -am "initial commit"
echo "done"

In [ ]:
git config --global credential.https://source.developers.google.com.helper gcloud.sh
git remote add google https://source.developers.google.com/p/hewagner-demos-2/r/acm_staging
git push --all google

If you want you can have a look at it in the browser by opening the following link:

In [ ]:
PROJECT_ID=$(gcloud config get-value core/project)
echo "https://source.cloud.google.com/$PROJECT_ID/$ACM_NAME"

![](https://github.com/Hermi999/bce/blob/master/img/cloudsourcerepo.png?raw=true)

What we want is that every time we do a commit to the master branch, our policies get automatically applied to the access context manager. 

For this we use [Cloud Build](https://cloud.google.com/cloud-build/) which get's triggered by a commit:

In [ ]:
gcloud beta builds triggers create cloud-source-repositories \
    --repo=$ACM_NAME \
    --branch-pattern="master" \
    --build-config=cloudbuild.yaml

Now that we have our trigger we can make a small change and commit the change. This will trigger a new __Cloud build__ which will deploy our access level policies to __Access Context Manager__. 

Before we trigger the build, let's open Cloud build in the UI to see how it's executing the build once we trigger it:

In [ ]:
PROJECT_ID=$(gcloud config get-value core/project)
echo "https://console.cloud.google.com/cloud-build/builds?project=$PROJECT_ID"

After we execute the next step a new entry will appear in the table. You can click on the link in the "Build" column. 

In [ ]:
# update the ip of the hightrust level (just for making any change so that we can commit)
sed -i "s/80.110.0.0/80.111.0.0/g" CD/hightrust.yaml
git commit -am "change IP for hightrust policy"
git push --all google

Here is what you should see in the UI:
![](https://github.com/Hermi999/bce/blob/master/img/cloudbuildhistory.png?raw=true)

When you follow the link you see the details of the build:
![](https://github.com/Hermi999/bce/blob/master/img/cloudbuilddetails.png?raw=true)

We can also verify the result by looking into the UI for the access context manager:

In [ ]:
PROJECT_ID=$(gcloud config get-value core/project)
ORG_ID=$(gcloud projects describe $PROJECT_ID --format="value(parent.id)")
echo "https://console.cloud.google.com/security/access-level?organizationId=$ORG_ID"

We also test if we can update the new access levels:

In [ ]:
sed -i "s/80.111.0.0/80.110.0.0/g" CD/hightrust.yaml
sed -i "s/IT/CZ/g" CD/lowtrust.yaml
git commit -am "change IP of hightrust and replace Italy with Czech for lowtrust"
git push --all google

In the next step we will combine IAP and Access-context manager to restrict access to our web app.

## Grant access on the application

### Grant access without context-aware access
The next step is to grant access to the protected app. 
We can do this on an individual user or group level and grant access to just a single app or to all apps in the whole project. First let's add yourself:

In [ ]:
USER="h@hewagner.com"
gcloud iap web add-iam-policy-binding --member="user:$USER" --role='roles/iap.httpsResourceAccessor'

Wait about a minute and try to access the web app url:

In [ ]:
gcloud app browse

You should see a welcome message and your email address.

In the GUI it should look similar to this:
![alt](https://github.com/Hermi999/bce/blob/master/img/iap.png?raw=true)

Here are the links to have a look for yourself:

In [ ]:
PROJECT_ID=$(gcloud config get-value core/project)
ORG_ID=$(gcloud projects describe $PROJECT_ID --format="value(parent.id)")
echo "https://console.cloud.google.com/security/iap?project=$PROJECT_ID"
echo "https://console.cloud.google.com/security/access-level?orgonly=true&organizationId=$ORG_ID"

### Grant access with context-aware access

*__NOTE TO GOOGLERS:__ The Context-aware access part of this tutorial will not work in the __google.com GCP Organization__, as you do not have the rights to view/edit the access policies and levels.*


Replace the USER emails with your email addresses:

In [ ]:
USER1=h@hewagner.com
USER2=a@hewagner.com
LEV1=Low_Trust
LEV2=High_Trust
POL=$(gcloud access-context-manager policies list --organization=$ORG_ID --format="value(NAME)")
TITLE1="low trust"
TITLE2="high trust"
gcloud iap web add-iam-policy-binding --member="user:$USER1" \
  --role='roles/iap.httpsResourceAccessor' \
  --condition="expression=(\"accessPolicies/$POL/accessLevels/$LEV1\" in request.auth.access_levels)",title="$TITLE1"

gcloud iap web add-iam-policy-binding --member="user:$USER2" \
  --role='roles/iap.httpsResourceAccessor' \
  --condition="expression=(\"accessPolicies/$POL/accessLevels/$LEV2\" in request.auth.access_levels)",title="$TITLE2"

![alt](https://github.com/Hermi999/bce/blob/master/img/caa-2.png?raw=true)

## Setup BeyondCorp for GCE Apps

(https://cloud.google.com/iap/docs/tutorial-gce)

In this section, you will deploy a VM into Google Cloud that will run a simple __nginx__. webapp.
In this flow, the request proceeds as: User -> IAP -> GCE backend

We start with setting the environment variables:

In [ ]:
PROJECT_ID=$(gcloud config get-value core/project)
ZONE=europe-west1-b
REGION=${ZONE%-[a-z]}
VM_NAME=nginx-vm
VPC=default
VPC_SUBNET=default
VPC_SUBNET_CIDR=`gcloud compute networks subnets describe $VPC --region=$REGION --format="value(ipCidrRange)" --project $PROJECT_ID`

echo $PROJECT_ID
echo $ZONE
echo $REGION
echo $VPC_SUBNET_CIDR

### NAT Gateway
The Backend VM will NOT have external connectivity - So we need to set up a NAT Gateway to allow the VM pulling the nginx docker image:

In [ ]:
gcloud compute routers create nat-router \
    --network $VPC \
    --region $REGION

gcloud compute routers nats create nat-config \
    --router=nat-router \
    --auto-allocate-nat-external-ips \
    --nat-all-subnet-ip-ranges \
    --enable-logging --region=$REGION

### Create VM
The following steps starts a VM and runs nginx in a Container Optimized OS:

In [ ]:
gcloud compute instances create-with-container $VM_NAME \
  --container-image nginx:latest \
  --no-address \
  --no-service-account \
  --no-scopes \
  --zone=$ZONE --project=$PROJECT_ID

### Create unmanaged Instance Group and add VM

In [ ]:
gcloud compute instance-groups unmanaged create gcp-uig --zone=$ZONE --project=$PROJECT_ID

gcloud compute instance-groups unmanaged add-instances gcp-uig \
    --zone=$ZONE --project=$PROJECT_ID --instances=nginx-vm
    
gcloud compute instance-groups unmanaged set-named-ports gcp-uig \
    --zone=$ZONE --project=$PROJECT_ID --named-ports=http:80

### Reserve an (global) external IP

In [ ]:
gcloud compute addresses create iap-ip --global --project=$PROJECT_ID 

ADDRESS=`gcloud compute addresses describe iap-ip --global --format="value(address)" --project=$PROJECT_ID`
echo $ADDRESS

### Manipulate /etc/hosts
This tutorial we will not configure a real domain and a DNS server to resolve our external IP address. Instead we will later create a self-signed certificate for the domain __server.domain.com__. 

To resolve server.domain.com from this local JupyterLab VM we will simple add an entry to the /etc/hosts file. We can't do this here in the ipynb notebook, so you need to navigate in the menu bar to "File", "New", "Terminal". 

Then execute the field below, copy the result and insert it into the new terminal.

In [ ]:
echo "sudo su"
echo "echo $ADDRESS server.domain.com >> /etc/hosts"
echo "exit"

This added an entry to the hosts file. We can have a look at the results here:

In [ ]:
cat /etc/hosts

### Create Backend VIP

In [ ]:
gcloud compute addresses create gce-backend-vip --region=$REGION \
--purpose GCE_ENDPOINT --project=$PROJECT_ID --subnet=$VPC

gcloud compute addresses list --project=$PROJECT_ID

We can see that wee have 3 reserved IP addresses (2 external and 1 internal)

### Crate Firewall Rules and Health check

In [ ]:
gcloud compute firewall-rules create allow-gcp-lb-http --allow=tcp:80 \
    --source-ranges=130.211.0.0/22,35.191.0.0/16  --project=$PROJECT_ID

gcloud compute http-health-checks create http-basic-check --port 80 \
    --request-path="/"  --project=$PROJECT_ID

### Add instance group to backend service

In [ ]:
gcloud compute backend-services create gcp-map-backend-service --global \
    --protocol HTTP --port-name=http --http-health-checks http-basic-check --project=$PROJECT_ID

gcloud compute backend-services add-backend gcp-map-backend-service \
    --balancing-mode UTILIZATION \
    --max-utilization 0.8 \
    --capacity-scaler 1 \
    --instance-group gcp-uig --instance-group-zone $ZONE  --project=$PROJECT_ID --global


### Upload SSL Certificate and configure L7 Load Balancer

We will enable SSL on the L7 LoadBalancer and will NOT be using Google Managed SSL Certificates as that will require DNS resolution. In the following step, we will create a self-signed certificate and upload it:


In [ ]:
mkdir ~/bce/certs/
cd ~/bce/certs

# create self-signed cert
openssl req -x509 -newkey rsa:2048 -keyout server_key.pem -out server_crt.pem \
    -days 365 -nodes -subj '/CN=server.domain.com'
    
gcloud compute ssl-certificates create gcp-cert --certificate server_crt.pem \
    --private-key server_key.pem

### Deploy L7 Load Balancer

In [ ]:
gcloud compute url-maps create gcp-map --default-service gcp-map-backend-service --project=$PROJECT_ID

gcloud compute url-maps add-path-matcher gcp-map --path-matcher-name=gcp \
    --default-service gcp-map-backend-service --project=$PROJECT_ID

gcloud compute url-maps add-host-rule gcp-map --hosts=server.domain.com \
    --path-matcher-name=gcp --project=$PROJECT_ID --global 

gcloud compute url-maps remove-host-rule gcp-map --host=*

gcloud compute target-https-proxies create gcp-lb-proxy --url-map=gcp-map \
    --ssl-certificates=gcp-cert --global  --project=$PROJECT_ID
 
gcloud compute forwarding-rules create gcp-content-rule --address $ADDRESS \
    --global --target-https-proxy gcp-lb-proxy --ports 443  --project=$PROJECT_ID


Let's have a look at this in the Google Clound Console:
![alt](https://github.com/Hermi999/bce/blob/master/img/vm_lb.png?raw=true)

In [ ]:
cd ~/bce/certs
curl --cacert server_crt.pem -w "%{http_code}\n" \
     --resolve server.domain.com:$ADDRESS https://server.domain.com/

You should see the __"Welcome to nginx!"__ html and the status code 200 at the end.
This means that anyone can connect to the website. Now let's protect it with IAP.

### Create a new Oauth Client and Enable IAP

(https://cloud.google.com/iap/docs/programmatic-oauth-clients#creating_an_oauth_client)

You configure one OAuth client per app.

In [ ]:
BRAND=$(gcloud alpha iap oauth-brands list --format="value(name)")
CLIENT_NAME=iap-gce
gcloud alpha iap oauth-clients create $BRAND --display_name=$CLIENT_NAME

The next step we can not (yet) do from the console with the gcloud client. You need to insert the Client id and client secret from the previous step, execute the field below and then past the resulting link into a new browser window:

In [ ]:
CLIENT_ID=
SECRET=
echo "https://console.cloud.google.com/apis/credentials/oauthclient/$CLIENT_ID?project=$PROJECT_ID"


You are now on the Oauth Client page. Execute the next field and add the result as "Authorized redirect URI" and click save:

In [ ]:
echo "https://iap.googleapis.com/v1/oauth/clientIds/$CLIENT_ID:handleRedirect"

In [ ]:
SVC_NAME=gcp-map-backend-service

gcloud alpha iap web enable --resource-type=backend-services --oauth2-client-id=$CLIENT_ID \
    --oauth2-client-secret=$SECRET --service $SVC_NAME

Your VM-based web app is now protected via IAP!

If you've executed the previous chapters then you don't need to do anything further. If not (or if you want to allow other users access) then have a look at the chapter __[Grant access on the app](#Grant-access-on-the-application)__.

In [ ]:
echo "https://$ADDRESS"

## Setup BeyondCorp for GKE Apps

__[Work in progress]__

(https://cloud.google.com/iap/docs/enabling-kubernetes-howto)

In this section, you will deploy a GKE Cluster that will run a simple web service.
In this flow, the request proceeds as: User -> IAP -> GKE backend

__IAP__ is integrated through __Ingress for GKE__.

In a GKE cluster, incoming traffic is handled by HTTP(S) Load Balancing. The HTTP(S) load balancer is typically configured by the Kubernetes Ingress controller. The Ingress controller gets configuration information from a Kubernetes Ingress object that is associated with one or more Service objects. Each Service object holds routing information that is used to direct an incoming request to a particular Pod and port.

The Kubernetes Ingress controller reads configuration information from the BackendConfig and sets up the load balancer accordingly. A BackendConfig holds configuration information that is specific to Cloud Load Balancing, and enables you to define a separate configuration for each HTTP(S) Load Balancing backend service.

We start with setting up a GKE cluster with HTTP(s) Load Balancing via Ingress ([read all details here](https://cloud.google.com/kubernetes-engine/docs/tutorials/http-balancer))


### Create new GKE cluster

In [ ]:
gcloud config set project $PROJECT_ID
gcloud config set compute/zone europe-west1-b
CLU_NAME=iap-gke-cluster
gcloud container clusters create $CLU_NAME
echo "done"

Wait 2-3 minutes.
After the cluster is created we can retrieve the credentials to connect via __kubectl__:

In [ ]:
gcloud container clusters get-credentials $CLU_NAME

### Create a global static ip for the service

In [ ]:
GKE_IP_NAME="iap-gke-ip"
gcloud compute addresses create $GKE_IP_NAME --global
GKE_IP=$(gcloud compute addresses describe $GKE_IP_NAME --global --format="value(address)")
echo $GKE_IP

### Create a SSL Certificate

(https://cloud.google.com/kubernetes-engine/docs/concepts/ingress-xlb)

(https://cloud.google.com/kubernetes-engine/docs/how-to/ingress-multi-ssl)

You can create a self-signed certificate, or if you have a domain you can also use letsencrypt.org.
To keep it simple I will show you how to do it with a self-signed certificate.

In [ ]:
mkdir ~/bce/certs/
cd ~/bce/certs

# create self-signed cert
openssl req -x509 -newkey rsa:2048 -keyout gke_key.pem -out gke_crt.pem \
    -days 365 -nodes -subj '/CN=gke.domain.com'
    
gcloud compute ssl-certificates create gke-cert --certificate gke_crt.pem \
    --private-key gke_key.pem

In [ ]:
gcloud compute ssl-certificates list
gcloud compute ssl-certificates describe gke-cert

### Manipulate /etc/hosts
This tutorial we will not configure a real domain and a DNS server to resolve our external IP address. Instead we will later create a self-signed certificate for the domain __server.domain.com__. 

To resolve server.domain.com from this local JupyterLab VM we will simple add an entry to the /etc/hosts file. We can't do this here in the ipynb notebook, so you need to navigate in the menu bar to "File", "New", "Terminal". 

Then execute the field below, copy the result and insert it into the new terminal.

In [ ]:
echo "sudo su"
echo "echo $GKE_IP server.domain.com >> /etc/hosts"
echo "exit"

### Inspect k8s specs
Let's see what we will deploy to the cluster:

In [ ]:
cd ~/bce/samples/gke/load-balancing

echo "-----deployment-----"
cat web-deployment.yaml && echo "-----service-----"
cat web-service.yaml && echo "-----ingress-----"
cat web-ingress-static.yaml && echo "-----backend-config-----"
cat web-backend-config.yaml

Here a few things you should notice:
- The service references the backend-config via an beta.cloud.google.com annotation
- The Ingress references a static ip with a kubernetes.io annotation
- The Ingress references a pre-shared ssl certificate with an annotation
- The Ingress specifies the domain name 
- The backend-config references a secret (holds the app oauth creds)
- The backend-config can enable/disable the IAP. Atm it's disabled.

### Add static ip name to ingress spec

In [ ]:
sed -i "s/STATIC_IP_NAME/$GKE_IP_NAME/g" web-ingress-static.yaml

cat web-ingress-static.yaml

### Create a new Oauth Client

(https://cloud.google.com/iap/docs/programmatic-oauth-clients#creating_an_oauth_client)

You configure one OAuth client per app.

In [ ]:
BRAND=$(gcloud alpha iap oauth-brands list --format="value(name)")
CLIENT_NAME=iap-gke
gcloud alpha iap oauth-clients create $BRAND --display_name=$CLIENT_NAME

The next step we can not (yet) do from the console with the gcloud client. You need to insert the Client id and client secret from the previous step, execute the field below and then past the resulting link into a new browser window:

In [ ]:
CLIENT_ID=
SECRET=
echo "https://console.cloud.google.com/apis/credentials/oauthclient/$CLIENT_ID?project=$PROJECT_ID"

You are now on the Oauth Client page. Execute the next field and add the result as "Authorized redirect URI" and click save:

In [ ]:
echo "https://iap.googleapis.com/v1/oauth/clientIds/$CLIENT_ID:handleRedirect"

### Create a k8s secret holding the OAuth client id and client secret
This secret will be used the the backend-service k8s object.

In [ ]:
kubectl create secret generic gke-iap-secret --from-literal=client_id=$CLIENT_ID \
    --from-literal=client_secret=$SECRET

### Deploy the k8s resources

In [ ]:
cd ~/bce/samples/gke/load-balancing/
kubectl apply -f web-backend-config.yaml
kubectl apply -f web-deployment.yaml

In [ ]:
kubectl apply -f web-service.yaml

In [ ]:
kubectl apply -f web-ingress-static.yaml

Wait 5-10 min until ingress deployed the load balancer. 

After a few minutes we can send a request to the static ip and it should respond with __"Hello, world!"__

In [ ]:
cd ~/bce/certs
ls
curl --cacert gke_crt.pem -w "%{http_code}\n" \
     --resolve gke.domain.com:$GKE_IP https://gke.domain.com/

You can also open the site via the IP in the browser (you can't open the domain in the browser if you didn't change /etc/hosts/ on your local machine) an ignore the self-signed cert warning:

In [ ]:
echo "$GKE_IP"

### Activate IAP
Time to protect our new web app...

In [ ]:
cd ~/bce/samples/gke/load-balancing/
sed -i "s/false/true/g" web-backend-config.yaml
cat web-backend-config.yaml

In [ ]:
kubectl apply -f .

After applying the change we need to wait again for a few minutes to see the effect...


In [ ]:
cd ~/bce/certs
ls
curl --cacert gke_crt.pem -w "%{http_code}\n" \
     --resolve gke.domain.com:$GKE_IP https://gke.domain.com/

Open the web app in the browser:

In [ ]:
echo "$GKE_IP"

### Reset the changed yaml files (optional)

In [ ]:
sed -i "s/true/false/g" web-backend-config.yaml
sed -i "s/$GKE_IP_NAME/STATIC_IP_NAME/g" web-ingress-static.yaml
cat web-backend-config.yaml && echo ""
cat web-ingress-static.yaml

## Setup BeyondCorp for __internal__ on-premise Apps

__[Work in progress]__

You will set up two projects: 
- one to host IAP and 
- another that represents your 'onprem' network which you will connect using a VPN tunnel

The onprem system will not be exposed externally and will run an apache web server that will represent your __intranet site__. The IAP project will establish a secure connection on the backend to the simulated 'on-prem' network using low level ipsec VPN as well as Cloud VPN.
This setup is more complex to set up since this involves establishing a secure backend connection via VPN.
In this flow, the request proceeds as: user -> IAP -> IAP Connector -> VPN --> backend


## Setup BeyondCorp for __external__ on-premise Apps

__[Work in progress]__

In this section, you will run a VM "on-prem" that is __exposed externally__ but one that validates requests authorized by IAP. This mode is for the situation where you would like to use IAP but cannot modify the underlying code to perform IAP's user validation.
For this, you will run a simple Envoy on prem that will validate IAP’s headers only. Once validated, the request is allowed to proceed to the backend system.
In this flow, the request proceeds as: user -> IAP -> Internet NEG --> backend



## Setup BeyondCorp for TCP forwarding

__[Work in progress]__

In this section, you will ...


## Zero Trust Strategy

__[Work in progres...]__

In general we have 3 dimensions we care about when implementing BeyondCorp:
- What? (The application)
- Who? - Who can access the application? (users, groups, services)
- When? - In which context can they access the application? (only if ...)

It's rather simply to define the applications. E.g.:
+ Jira
+ Gitlab
+ Spinnaker (Prod, Staging, Dev)
+ SAP Web UI
+ HR System
+ Concur
+ GSuite
+ ...

For each of those apps we can now aggregate the different user types into __groups__:
+ Admins
+ Power Users
+ Users

We can define these general __context levels__:
1. High Trust (Admin access, PII & IP data)
2. Medium Trust (Need to know data)
3. Low Trust (all other)

Now let's vizualize this in a simple table:

![alt](https://github.com/Hermi999/bce/blob/master/img/bce_table.png?raw=true)


If we want we can turn this table (automatically) into yaml files which we then can use in a CD pipeline for configuring the Identity-aware proxy.